In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src import supabase_client
from src.update_pipeline import update_training_week
from src.types.update_pipeline import ExeType
from src.utils import get_last_sunday
from src import auth_manager

from freezegun import freeze_time
import os
from datetime import timedelta

In [ ]:
@freeze_time(f"{get_last_sunday()} 18:00:00")
def frozen_update_training_week_new_week(user):
    return update_training_week(user, ExeType.NEW_WEEK)

for user in supabase_client.list_users(debug=True):
    auth_manager.authenticate_athlete(user.athlete_id)
    try:
        response = frozen_update_training_week_new_week(user)
        print(response)
    except Exception as e:
        print(e)

In [ ]:
user = supabase_client.get_user(os.environ["JAMIES_ATHLETE_ID"])
auth_manager.authenticate_athlete(user.athlete_id)
response = update_training_week(user, ExeType.MID_WEEK)
print(response)